In [ ]:
from urllib.request import Request, urlopen
from http.cookiejar import CookieJar
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import requests
from datetime import date
import time as t
import pandas as pd
from http.cookiejar import CookieJar
import random
import requests
import re

# Scrapper 

In [ ]:
link = "insert here"

In [ ]:
# Parameters
## Setting up the link to scrap
start_link = link
list_link = []
loop_inf = -1
loop_sup = 1
x = random.randint(2, 5)
kicker = 0


while loop_inf < loop_sup :
    
    same_link = False
    
    if kicker == 500 or same_link == True:
        loop_inf = loop_inf
    else:
        loop_inf = loop_inf + 1 
        print("added 1")
        
    
    ## Get the data from the starting page - Using an API to bypass security 
    params = {'api_key': 'XXX', 'url': start_link, 'params': "...."}
    response = requests.get('XXXX', params=params)
    webpage = response.text
    soup = BeautifulSoup(webpage, 'html5lib')
    

    if response.status_code == 200 :
        print("ok, proceed to grab info")
        print(response.status_code)

        if len(soup.find_all('a', attrs={'class' : 'absolute inset-0'})) != 0:
                # Get ad links
                for item in soup.find_all('a', attrs={'class' : 'absolute inset-0'}):
                    ad_link = item.get('href')
                    list_link.append(ad_link)
                print("Added links")
                

                # Get next page  
                for item in soup.find_all('div', attrs={'class' : 'flex w-full justify-center'}):
                    for a in item.find_all("a"):
                        last_link = a.get("href")
                        
                        loop_sup = int(last_link[-1])
                        test_link = str("https://www.leboncoin.fr"+str(last_link))
                        if start_link != test_link:
                            start_link = "https://www.leboncoin.fr"+str(last_link)
                            kicker = 0
                        else :
                            print("Same link")
                            kicker = 0
                            same_link = True
                            t.sleep(x)
                            continue
                            
                    print("new start_link : "+ start_link)
           
        x = random.randint(5, 10)
        t.sleep(x)
        
    elif response.status_code == 500 :
        kicker = 500
        print("scrapper encountered a problem")
        print(response.status_code)
        x = random.randint(5, 10)
        t.sleep(x)
        continue 
        

    print("loop_inf : " + str(loop_inf))
    print("loop_sup : " + str(loop_sup))
    
print("Scraper finished")

In [ ]:
# Sort out doubles
list_link = list(set(list_link))

In [ ]:
len(list_link)

In [ ]:
# Parameters
## Setting up the link to scrap


count = 0
list_ad_to_store = []
itera = 0
columns = ["Type", "surface_h", "surface_t", "piece", "chambre", "prix", "id_ad"]
df = pd.DataFrame([], columns = columns)
list_dim = []
list_ad_id = []
list_time = []
list_desc = []



   
for ad_link in list_link:
    if itera < len(list_link):
        ad_link_scrap = "https://www.leboncoin.fr"+str(ad_link)
        start_link = ad_link_scrap
        ## Get the data from the starting page
        params = {'api_key': '39dc9cd4f54af1e852b1300b785c31d1', 'url': start_link, 'device_type': 'desktop',  "ultra_premium" : "true"}
        response = requests.get('http://api.scraperapi.com/', params=urlencode(params))
        webpage = response.text
        soup = BeautifulSoup(webpage, 'html5lib')

        if response.status_code == 500:
            print("Error 500", response.status_code)
            x = random.randint(15, 35)
            t.sleep(x)
            continue

        else :
            
            for price in soup.find_all('p', attrs={'class' : 'text-headline-2'}):
                pricing = str(price.text)
            
            for place in soup.find_all('a', attrs={'class' : 'text-body-1'}):
                place_ad = str(place.text)
                


            for desc in soup.find_all('p', attrs={'class' : 'text-body-1 truncate font-bold'}):
                if desc.text == "":
                    list_desc.append(" ")
                    list_ad_id.append(re.sub(r'[^\d]', '', ad_link_scrap))
                    list_time.append(date.today().strftime("%Y-%m-%d-%H:%M:%S"))
                else:
                    list_desc.append(desc.text)
                    list_ad_id.append(re.sub(r'[^\d]', '', ad_link_scrap))
                    list_time.append(date.today().strftime("%Y-%m-%d-%H:%M:%S"))
            
            
            for dim in soup.find_all('p'):
                class_string = dim.get('class')
                if class_string == ['mb-sm', 'truncate', 'text-caption', 'opacity-dim-1'] or \
                   class_string == ['mb-sm', 'text-caption', 'opacity-dim-1', 'truncate']:

            #for dim in soup.find_all('p', attrs={'class' : 'mb-sm truncate text-caption opacity-dim-1'}):
                    list_dim.append(dim.text)
                    print(str(itera)+": "+ dim.text)
                
            #count = count + 1

            pricing = re.sub(r'[^\d]', '', pricing)
            list_desc.append(pricing)
            list_ad_id.append(re.sub(r'[^\d]', '', ad_link_scrap))
            list_time.append(date.today().strftime("%Y-%m-%d-%H:%M:%S"))
            list_dim.append("price")
            
            list_desc.append(place.text)
            print(str(place.text))
            list_ad_id.append(re.sub(r'[^\d]', '', ad_link_scrap))
            list_time.append(date.today().strftime("%Y-%m-%d-%H:%M:%S"))
            list_dim.append("place")
            
            #df_temp = pd.DataFrame([list_desc], columns = columns)
            #df = pd.concat([df, df_temp], ignore_index = True)
            itera = itera + 1
            x = random.randint(15, 35)
            t.sleep(x)

            
            print(start_link)
            print("done")
    else:
        print("Scraper is finished")
        break

# Data Transformation

In [ ]:
df = pd.DataFrame({
    'ad_id': list_ad_id,
    'time': list_time,
    'dim': list_dim,
    'desc': list_desc,
})


df_1 = df.copy()

df_1.replace("Disponible à partir de", "dispo", inplace = True)
df_1.replace("Honoraires inclus", "Hono_inclus", inplace = True)
df_1.replace("Nombre de chambres", "nomb_chambre", inplace = True)
df_1.replace("Nombre de pièces", "nomb_piece", inplace = True)
df_1.replace("Nombre de salles d'eau", "nomb_sal_eau", inplace = True)
df_1.replace("Nombre de salles de bain", "nomb_sal_bain", inplace = True)
df_1.replace("Surface habitable", "hab_m2", inplace = True)
df_1.replace("Surface totale du terrain", "terr_m2", inplace = True)
df_1.replace("Type de bien", "type_bien", inplace = True)
df_1.replace("Type de vente", "type_vente", inplace = True)
df_1.replace("Nature du bien", "nature_bien", inplace = True)
df_1.replace("Charges honoraires", "honoraire", inplace = True)
df_1.replace("Ascenseur", "ascenceur", inplace = True)
df_1.replace("Exposition", "expo", inplace = True)
df_1.replace("Référence", "ref", inplace = True)

pivoted = df_1.pivot(index=['ad_id', 'time'], columns='dim', values='desc')
pivoted = pivoted.reset_index()

if 'nature_bien' in pivoted.columns:
    pivoted['nature_bien'] = pivoted['nature_bien'].fillna("Aucun")
else :
    pass

if 'nature_bien' in pivoted.columns:
    pivoted['nature_bien'] = pivoted['nature_bien'].fillna("Aucun")
else :
    pass

if 'nomb_piece' in pivoted.columns:
    pivoted['nomb_piece'] = pivoted['nomb_piece'].fillna(0)

else :
    pass

if 'type_vente' in pivoted.columns:
    pivoted['type_vente'] =  pivoted["type_vente"].fillna("Aucun")
else :
    pass

if 'hab_m2' in pivoted.columns :
    pivoted['hab_m2'] = pivoted['hab_m2'].fillna(0).str.extract(r'(\d+)').fillna(0).astype(int)
else:
    pass

if 'terr_m2' in pivoted.columns :
    pivoted['terr_m2'] = pivoted['terr_m2'].fillna(0).str.extract(r'(\d+)').fillna(0).astype(int)
else:
    pass

if 'nomb_chambre' in pivoted.columns :
    pivoted['nomb_chambre'] = pivoted['nomb_chambre'].fillna(0).str.extract(r'(\d+)').fillna(0).astype(int)
else:
    pass

pivoted['post_code'] = pivoted['place'].str[-6:]
pivoted['place'] =  pivoted['place'].str[:-6]




final = pivoted[["ad_id", "time", "type_bien", "hab_m2", "terr_m2","nomb_piece", "nomb_chambre","price", "place","post_code"]].copy()

In [ ]:
df_price_follow_up = pivoted[["ad_id", "time", "price"]].copy()
df_price_follow_up["ad_id"] = pd.to_numeric(df_price_follow_up["ad_id"], errors='coerce')
df_price_follow_up["price"] = pd.to_numeric(df_price_follow_up["price"], errors='coerce')
df_price_follow_up['time'] = pd.to_datetime(df_price_follow_up['time'], format="%Y-%m-%d-%H:%M:%S", errors='coerce')

final['time'] = pd.to_datetime(final['time'], format="%Y-%m-%d-%H:%M:%S", errors='coerce')
final["ad_id"] = pd.to_numeric(final["ad_id"], errors='coerce')
final["price"] = pd.to_numeric(final["price"], errors='coerce')
final["hab_m2"] = pd.to_numeric(final["hab_m2"], errors='coerce')
final["terr_m2"] = pd.to_numeric(final["terr_m2"], errors='coerce')
final["nomb_piece"] = pd.to_numeric(final["nomb_piece"], errors='coerce')
final["nomb_chambre"] = pd.to_numeric(final["nomb_chambre"], errors='coerce')
final['ad_id'] = final['ad_id'].astype(str)

final = final.dropna(subset=['type_bien'])

# Load



In [ ]:
import pyodbc

In [ ]:
server = 'XXXXX'
database = 'XXX'
username = 'XXXX'
password = 'XXXX'

In [ ]:
connection_string = (
    f'DRIVER={{ODBC Driver 18 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password};'
    'Encrypt=yes;'
    'TrustServerCertificate=no;'
    'Connection Timeout=30;'
)

In [ ]:
try:
    conn = pyodbc.connect(connection_string)
    print("✅ Connected to Azure SQL successfully.")
    conn.close()
except Exception as e:
    print("❌ Connection failed:", e)

# Insert price follow up data

In [ ]:
# Create connection and cursor
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Example: inserting one row
sql = """
INSERT INTO price_follow_up (ad_id, time, price)
VALUES (?, ?, ?)
"""
rows_to_insert = []

for _, row in df_price_follow_up.iterrows():
    rows_to_insert.append((
        row['ad_id'],
        row['price'],
        row['time'],
        row['ad_id'],  # doublé pour le WHERE
        row['time']   # doublé pour le WHERE
    ))
    
    
sql = """
INSERT INTO price_follow_up (ad_id, price, time)
SELECT ?, ?, ?
WHERE NOT EXISTS (
    SELECT 1 FROM price_follow_up WHERE ad_id = ? AND time = ?
);
"""

cursor.executemany(sql, rows_to_insert)
conn.commit()

print("✅ Rows inserted successfully.")
conn.close()

# Inserting ads details

In [ ]:
# Create connection and cursor
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

rows_to_insert = []

for _, row in final.iterrows():
    rows_to_insert.append((
        row['ad_id'],
        row['time'],
        row['type_bien'],
        row['hab_m2'], 
        row['terr_m2'], 
        row['nomb_piece'], 
        row['nomb_chambre'], 
        row['price'], 
        row['place'], 
        row['post_code']
    ))
    
    
sql = """
MERGE desc_ad AS target
USING (SELECT ? AS ad_id, ? AS time, ? AS type_bien, ? AS hab_m2, ? AS terr_m2,
              ? AS nomb_piece, ? AS nomb_chambre, ? AS price, ? AS place, ? AS post_code) AS source
ON target.ad_id = source.ad_id

WHEN MATCHED THEN 
    UPDATE SET 
        time = source.time,
        type_bien = source.type_bien,
        hab_m2 = source.hab_m2,
        terr_m2 = source.terr_m2,
        nomb_piece = source.nomb_piece,
        nomb_chambre = source.nomb_chambre,
        price = source.price,
        place = source.place,
        post_code = source.post_code

WHEN NOT MATCHED THEN 
    INSERT (ad_id, time, type_bien, hab_m2, terr_m2, nomb_piece, nomb_chambre, price, place, post_code)
    VALUES (source.ad_id, source.time, source.type_bien, source.hab_m2, source.terr_m2,
            source.nomb_piece, source.nomb_chambre, source.price, source.place, source.post_code);
"""

cursor.executemany(sql, rows_to_insert)
conn.commit()

print("✅ Rows inserted successfully.")
conn.close()